In [13]:
import torchxrayvision as xrv

from skimage import color
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

fatal: destination path 'covid-chestxray-dataset' already exists and is not an empty directory.


In [3]:
d = xrv.datasets.COVID19_Dataset(imgpath="covid-chestxray-dataset/images/",csvpath="covid-chestxray-dataset/metadata.csv")

In [4]:
d.csv.head()

,index,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,...,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29,offset_day_int
0,0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,0.0
1,1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,3.0
2,2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,5.0
3,3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,6.0
4,4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,...,"Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN,0.0


In [5]:
split_tags = [ val.split('/') for val in d.csv['finding'] ]

In [6]:
all_tags = set()

for split_tag in split_tags:
    for val in split_tag:
        all_tags.add(val)

all_tags = list(all_tags)
print(all_tags)

['Aspergillosis', 'Fungal', 'Viral', 'Nocardia', 'No Finding', 'Streptococcus', 'Aspiration', 'COVID-19', 'Klebsiella', 'MERS-CoV', 'Lipoid', 'Bacterial', 'Legionella', 'H1N1', 'Mycoplasma', 'MRSA', 'Pneumocystis', 'Staphylococcus', 'Pneumonia', 'Tuberculosis', 'Influenza', 'Herpes ', 'SARS', 'Varicella', 'Chlamydophila']


In [7]:
tag_index = {'Bacterial':1, 'Klebsiella':1, 'H1N1':2, 'SARS':0, 'Aspergillosis':1, 'Staphylococcus':1, 'Herpes ':0, 'Varicella':0, 'Viral':0, 'Pneumonia':0, 'Mycoplasma':1, 'Streptococcus':1, 'Nocardia':1, 'MERS-CoV':0, 'MRSA':2, 'Influenza':0, 'Lipoid':2, 'Pneumocystis':1, 'No Finding':2, 'Chlamydophila':1, 'Fungal':1, 'Aspiration':2, 'Legionella':1, 'Tuberculosis':2, 'COVID-19':0}

In [8]:
data = d.csv[['finding', 'filename']]

In [9]:
data.head()

,finding,filename
0,Pneumonia/Viral/COVID-19,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...
1,Pneumonia/Viral/COVID-19,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...
2,Pneumonia/Viral/COVID-19,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...
3,Pneumonia/Viral/COVID-19,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...
4,Pneumonia/Viral/COVID-19,nejmc2001573_f1a.jpeg


In [10]:
bitmasks = []

for i, row in data.iterrows():
    bitmask = 0
    
    for val in [tag_index[extracted_tag] for extracted_tag in row['finding'].split('/') ]:
        bitmask = bitmask | (1 << val)

    bitmasks.append(bitmask)

data['bitmasks'] = bitmasks
data.head()
    

,finding,filename,bitmasks
0,Pneumonia/Viral/COVID-19,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,1
1,Pneumonia/Viral/COVID-19,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,1
2,Pneumonia/Viral/COVID-19,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,1
3,Pneumonia/Viral/COVID-19,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,1
4,Pneumonia/Viral/COVID-19,nejmc2001573_f1a.jpeg,1


In [11]:
print(data['bitmasks'].unique())

[1 3 4 5 7]


In [30]:
def read_image(filename):
    image_as_np = io.imread("./covid-chestxray-dataset/images/" + filename)
    if len(image_as_np.shape) == 3:
        try:
            image_as_np = color.rgb2gray(image_as_np)
        except:
            return None
    RESOLUTION = 10
    return resize(image_as_np, (RESOLUTION, RESOLUTION), anti_aliasing=True).flatten()

In [32]:
X = []
y = []

for i, val in data.iterrows():
    image = read_image(val['filename'])
    print(i)
    print(val['filename'])
    if image is None:
        continue
    X.append(image)
    y.append(val['bitmasks'])

0
auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
1
auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
2
auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
3
auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
4
nejmc2001573_f1a.jpeg
5
nejmc2001573_f1b.jpeg
6
ARDSSevere.png
7
lancet-case2a.jpg
8
lancet-case2b.jpg
9
SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg
10
SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg
11
SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg
12
SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7.jpeg
13
SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12.jpeg
14
SARS-10.1148rg.242035193-g04mr34g05x-Fig5-day9.jpeg
15
SARS-10.1148rg.242035193-g04mr34g07a-Fig7a-day5.jpeg
16
SARS-10.1148rg.242035193-g04mr34g07b-Fig7b-day12.jpeg
17
SARS-10.1148rg.242035193-g04mr34g09a-Fig9a-day17.jpeg
18
SARS-10.1148rg.242035193-g04mr34g09b-Fig9b-day19.jpeg
19
SARS-10.1148rg.242035193-g04mr34g09c-Fig9c-

KeyboardInterrupt: 